<a href="https://colab.research.google.com/github/suparnobanerjee/tutorial/blob/main/zephyr_7b_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install transformers
!pip install torch
!pip install accelerate
!pip install optimum
!pip install auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.9 MB/s eta 0:00:00
  Created wheel for optimum: filename=optimum-1.13.2-py3-none-any.whl size=395599 sha256=e218cc252188f5e7404370346da805685b75c7c31bd87b5cfa1928dd539fa9d6
  Stored in directory: /root/.cache/pip/wheels/6e/b7/2c/79405d98f0943373d8546daeae25a3d377f7659ca0cbe48699
Successfully built optimum


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/zephyr-7B-alpha-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Write a python code which can calculate the nth fibonacci number"
prompt_template=f'''<|system|>
</s>
<|user|>
{prompt}</s>
<|assistant|>
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])



*** Generate:
<s> <|system|>
</s> 
<|user|>
Write a python code which can calculate the nth fibonacci number</s> 
<|assistant|>
Here's the Python code to calculate the nth Fibonacci number:

```python
def fibonacci(n):
    # Initialize first two numbers in the series
    a = 0
    b = 1

    # Check base cases
    if n == 0:
        return a
    elif n == 1:
        return b
    elif n < 0:
        raise ValueError("Fibonacci numbers are not defined for negative indices")

    # Calculate nth Fibonacci number
    for i in range(2, n):
        # Calculate the next number in the series
        c = a + b
        # Update the previous two numbers in the series
        a, b = b, c

    # Return the nth Fibonacci number
    return b
```

Here's an example of using this function:

```python
>>> fibonacci(5)
125
```

The output is the 5th Fibonacci number, which is 125.</s>
*** Pipeline:
<|system|>
</s>
<|user|>
Write a python code which can calculate the nth fibonacci number</s>
<|assistant